# Dictionary learning

## Initialize data

In [4]:
# initialize data 
###
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# plot table
import texttable as tt

def plotTable(headers,
              rows,
              valign=True,
              column_width=False,
              no_deco=False,
              precision=1):
    tab = tt.Texttable()
    if len(headers) > 0:
        tab.header(headers)
    tab.set_precision(precision)
    for row in rows:
        tab.add_row(row)

    if valign and len(headers) > 0:
        tab.set_cols_valign(['m'] * len(headers))

    if column_width:
        columns = [0] * len(rows[0])
        for row in rows:
            for i, cell in enumerate(row):
                if isinstance(row[i], str):
                    columns[i] = max(
                        columns[i],
                        max([len(line) for line in row[i].split('\n')]))
        for i, head in enumerate(headers):
            columns[i] = max(columns[i], len(head))
        tab.set_cols_width(columns)

    if no_deco:
        tab.set_deco(tt.Texttable.HEADER | tt.Texttable.VLINES)

    s = tab.draw()
    return s

def data_from_file(file_name,case):
    #case = 1-> Id from file. Iq in 0->220 > 'i_direct_flux'
    #case = 2 -> Iq from file. Id in 0->220 > 'j_q_flux'
    data = np.loadtxt(file_name)
    ids = data[0, :]    
    dfs = data[1:, :]
    iqs = list(range(220, -1, -20))
    if case == 1:
        xLabel = 'd-axis current - Id [A]'
        yLabel = 'Direct flux linkage [Wb]'
        Is = 'Iq '
    elif case == 2:
        xLabel = 'q-axis current -Iq [A]'
        yLabel = 'Quadratic flux [Wb]'  
        Is = 'Id'
    return ids, dfs, iqs, xLabel, yLabel, Is


ids, dfs, iqs, xLabel, yLabel, Is = data_from_file('i_direct_flux',1 )


In [29]:
from sklearn.decomposition import MiniBatchDictionaryLearning as MBDL
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize as norm


X = dfs.T
train_data, test_data = train_test_split(X, test_size=0.4, random_state=0)

model = MBDL(n_components=1024, n_iter=100)
dex = model.fit(train_data).components_
dex_norm  = norm(dex)

from sklearn.decomposition import SparseCoder

coder = SparseCoder(dictionary = dex_norm, transform_n_nonzero_coefs=20,
                    transform_alpha=None, transform_algorithm="omp")

result = coder.transform(test_data)

print(test_data)
print("after DEX")
print(result)

[[-1.9975635e-05  8.6870954e-02  1.3827379e-01  1.6177552e-01
   1.7549161e-01  1.8505898e-01  1.9207943e-01  1.9772419e-01
   2.0235362e-01  2.0633254e-01  2.0993233e-01  2.1321770e-01]
 [ 0.0000000e+00  1.2081221e-01  1.6600134e-01  1.8052537e-01
   1.8979866e-01  1.9644351e-01  2.0169338e-01  2.0594191e-01
   2.0961936e-01  2.1291724e-01  2.1582123e-01  2.1849489e-01]
 [-4.2824158e-06  7.7490572e-02  1.2792570e-01  1.5404485e-01
   1.6895250e-01  1.7935817e-01  1.8718516e-01  1.9334807e-01
   1.9843112e-01  2.0274365e-01  2.0659894e-01  2.1012219e-01]
 [-3.6718047e-05  1.1173969e-01  1.6059269e-01  1.7756097e-01
   1.8786705e-01  1.9509484e-01  2.0068334e-01  2.0514289e-01
   2.0901493e-01  2.1237167e-01  2.1538032e-01  2.1814800e-01]
 [ 1.0897785e-05  6.9223888e-02  1.1804044e-01  1.4636957e-01
   1.6227744e-01  1.7342253e-01  1.8193032e-01  1.8860431e-01
   1.9415298e-01  1.9883636e-01  2.0297266e-01  2.0674102e-01]]
after DEX
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\dict_learning.py:160: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  copy_Xy=copy_cov).T
